In [48]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import functions as F

import os
import pandas as pd
from dotenv import load_dotenv

Configuration

In [49]:
load_dotenv()
credentials_location = os.getenv('GCP_CREDENTIALS_LOCATION')
gcp_bucket_name = os.getenv('GCP_BUCKET')

In [3]:
conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", "/Users/Manu/lib/gcs-connector-hadoop3-2.2.5.jar") \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

Context

In [4]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", 'true')

22/04/11 12:41:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Session

In [5]:
spark = SparkSession.builder \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.11:0.12.2", conf=sc.getConf()) \
    .getOrCreate()

Reading data from google cloug storage

In [81]:
cot = spark.read \
    .text(f'gs://{gcp_bucket_name}/raw/*')

In [57]:
cot.show(5)

+--------------------+
|               value|
+--------------------+
|"Market_and_Excha...|
|"CANADIAN DOLLAR ...|
|"CANADIAN DOLLAR ...|
|"CANADIAN DOLLAR ...|
|"CANADIAN DOLLAR ...|
+--------------------+
only showing top 5 rows



In [8]:
cot.count()

24140

In [9]:
cot.collect()[:2]

[Row(value='"Market_and_Exchange_Names","As_of_Date_In_Form_YYMMDD","Report_Date_as_YYYY-MM-DD","CFTC_Contract_Market_Code","CFTC_Market_Code","CFTC_Region_Code","CFTC_Commodity_Code","Open_Interest_All","Dealer_Positions_Long_All","Dealer_Positions_Short_All","Dealer_Positions_Spread_All","Asset_Mgr_Positions_Long_All","Asset_Mgr_Positions_Short_All","Asset_Mgr_Positions_Spread_All","Lev_Money_Positions_Long_All","Lev_Money_Positions_Short_All","Lev_Money_Positions_Spread_All","Other_Rept_Positions_Long_All","Other_Rept_Positions_Short_All","Other_Rept_Positions_Spread_All","Tot_Rept_Positions_Long_All","Tot_Rept_Positions_Short_All","NonRept_Positions_Long_All","NonRept_Positions_Short_All","Change_in_Open_Interest_All","Change_in_Dealer_Long_All","Change_in_Dealer_Short_All","Change_in_Dealer_Spread_All","Change_in_Asset_Mgr_Long_All","Change_in_Asset_Mgr_Short_All","Change_in_Asset_Mgr_Spread_All","Change_in_Lev_Money_Long_All","Change_in_Lev_Money_Short_All","Change_in_Lev_Money_S

1. All colums have been combined into one column called `value`
2. Need to split the column

Extract the column names from the row object

In [26]:
col_row = cot.collect()[0].asDict()['value']
print(col)

"Market_and_Exchange_Names","As_of_Date_In_Form_YYMMDD","Report_Date_as_YYYY-MM-DD","CFTC_Contract_Market_Code","CFTC_Market_Code","CFTC_Region_Code","CFTC_Commodity_Code","Open_Interest_All","Dealer_Positions_Long_All","Dealer_Positions_Short_All","Dealer_Positions_Spread_All","Asset_Mgr_Positions_Long_All","Asset_Mgr_Positions_Short_All","Asset_Mgr_Positions_Spread_All","Lev_Money_Positions_Long_All","Lev_Money_Positions_Short_All","Lev_Money_Positions_Spread_All","Other_Rept_Positions_Long_All","Other_Rept_Positions_Short_All","Other_Rept_Positions_Spread_All","Tot_Rept_Positions_Long_All","Tot_Rept_Positions_Short_All","NonRept_Positions_Long_All","NonRept_Positions_Short_All","Change_in_Open_Interest_All","Change_in_Dealer_Long_All","Change_in_Dealer_Short_All","Change_in_Dealer_Spread_All","Change_in_Asset_Mgr_Long_All","Change_in_Asset_Mgr_Short_All","Change_in_Asset_Mgr_Spread_All","Change_in_Lev_Money_Long_All","Change_in_Lev_Money_Short_All","Change_in_Lev_Money_Spread_All","

Split the `col_row` to get a list of columns and remove the extra quotation marks

In [41]:
cols_cleaned = []

cols = col.split(",")
for cl in cols:
    cols_cleaned.append(cl[1:-1])
    
for key, val in enumerate(cols_cleaned):
    print(f'{key}, {val} \n')

0, Market_and_Exchange_Names 

1, As_of_Date_In_Form_YYMMDD 

2, Report_Date_as_YYYY-MM-DD 

3, CFTC_Contract_Market_Code 

4, CFTC_Market_Code 

5, CFTC_Region_Code 

6, CFTC_Commodity_Code 

7, Open_Interest_All 

8, Dealer_Positions_Long_All 

9, Dealer_Positions_Short_All 

10, Dealer_Positions_Spread_All 

11, Asset_Mgr_Positions_Long_All 

12, Asset_Mgr_Positions_Short_All 

13, Asset_Mgr_Positions_Spread_All 

14, Lev_Money_Positions_Long_All 

15, Lev_Money_Positions_Short_All 

16, Lev_Money_Positions_Spread_All 

17, Other_Rept_Positions_Long_All 

18, Other_Rept_Positions_Short_All 

19, Other_Rept_Positions_Spread_All 

20, Tot_Rept_Positions_Long_All 

21, Tot_Rept_Positions_Short_All 

22, NonRept_Positions_Long_All 

23, NonRept_Positions_Short_All 

24, Change_in_Open_Interest_All 

25, Change_in_Dealer_Long_All 

26, Change_in_Dealer_Short_All 

27, Change_in_Dealer_Spread_All 

28, Change_in_Asset_Mgr_Long_All 

29, Change_in_Asset_Mgr_Short_All 

30, Change_in_Asset_

Generating new columns

In [82]:
cot_split = cot.select("*") # Copy the data frame

split_cols = F.split(cot_split['value'], ",")

for key, value in enumerate(cols_cleaned):
    cot_split = cot_split.withColumn(value, split_cols.getItem(key))

cot_split.select(cols_cleaned[:4]).show()

+-------------------------+-------------------------+-------------------------+-------------------------+
|Market_and_Exchange_Names|As_of_Date_In_Form_YYMMDD|Report_Date_as_YYYY-MM-DD|CFTC_Contract_Market_Code|
+-------------------------+-------------------------+-------------------------+-------------------------+
|     "Market_and_Excha...|     "As_of_Date_In_Fo...|     "Report_Date_as_Y...|     "CFTC_Contract_Ma...|
|     "CANADIAN DOLLAR ...|                   211228|               2021-12-28|                   090741|
|     "CANADIAN DOLLAR ...|                   211221|               2021-12-21|                   090741|
|     "CANADIAN DOLLAR ...|                   211214|               2021-12-14|                   090741|
|     "CANADIAN DOLLAR ...|                   211207|               2021-12-07|                   090741|
|     "CANADIAN DOLLAR ...|                   211130|               2021-11-30|                   090741|
|     "CANADIAN DOLLAR ...|                   

Add index column, use index column to filter out the first row, drop the first column and index column

In [97]:
# Add index column
cot_split = cot_split.withColumn('index', F.monotonically_increasing_id())

# filter out first column and drop value and index columns
cot_split = cot_split.filter(cot_split['index'] >= 1) \
    .drop('value', 'index')

cot_split.select(cols_cleaned[:4]).show()

+-------------------------+-------------------------+-------------------------+-------------------------+
|Market_and_Exchange_Names|As_of_Date_In_Form_YYMMDD|Report_Date_as_YYYY-MM-DD|CFTC_Contract_Market_Code|
+-------------------------+-------------------------+-------------------------+-------------------------+
|     "CANADIAN DOLLAR ...|                   211019|               2021-10-19|                   090741|
|     "CANADIAN DOLLAR ...|                   211012|               2021-10-12|                   090741|
|     "CANADIAN DOLLAR ...|                   211005|               2021-10-05|                   090741|
|     "CANADIAN DOLLAR ...|                   210928|               2021-09-28|                   090741|
|     "CANADIAN DOLLAR ...|                   210921|               2021-09-21|                   090741|
|     "CANADIAN DOLLAR ...|                   210914|               2021-09-14|                   090741|
|     "CANADIAN DOLLAR ...|                   

In [99]:
cot_split.printSchema()

root
 |-- Market_and_Exchange_Names: string (nullable = true)
 |-- As_of_Date_In_Form_YYMMDD: string (nullable = true)
 |-- Report_Date_as_YYYY-MM-DD: string (nullable = true)
 |-- CFTC_Contract_Market_Code: string (nullable = true)
 |-- CFTC_Market_Code: string (nullable = true)
 |-- CFTC_Region_Code: string (nullable = true)
 |-- CFTC_Commodity_Code: string (nullable = true)
 |-- Open_Interest_All: string (nullable = true)
 |-- Dealer_Positions_Long_All: string (nullable = true)
 |-- Dealer_Positions_Short_All: string (nullable = true)
 |-- Dealer_Positions_Spread_All: string (nullable = true)
 |-- Asset_Mgr_Positions_Long_All: string (nullable = true)
 |-- Asset_Mgr_Positions_Short_All: string (nullable = true)
 |-- Asset_Mgr_Positions_Spread_All: string (nullable = true)
 |-- Lev_Money_Positions_Long_All: string (nullable = true)
 |-- Lev_Money_Positions_Short_All: string (nullable = true)
 |-- Lev_Money_Positions_Spread_All: string (nullable = true)
 |-- Other_Rept_Positions_Long_

In [100]:
cot_split.write.parquet(f'gs://{gcp_bucket_name}/cleaned/pq')